In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import ComplementNB, GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.impute import SimpleImputer

In [2]:
import numpy as np

In [3]:
years = [2018, 2019, 2020, 2021, 2022]
dfs = []
for year in years:
    df = pd.read_csv(f'https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_{year}.csv')
    df['year'] = year
    dfs.append(df)
final_df = pd.concat(dfs)

In [4]:
final_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year
0,2018-M020,Brisbane,Hard,32,A,20180101,271,105992,NaN,NaN,...,33.0,19.0,14.0,1.0,4.0,47.0,1010.0,52.0,909.0,2018
1,2018-M020,Brisbane,Hard,32,A,20180101,272,111577,NaN,NaN,...,25.0,7.0,9.0,7.0,11.0,54.0,890.0,94.0,593.0,2018
2,2018-M020,Brisbane,Hard,32,A,20180101,273,104797,NaN,NaN,...,37.0,29.0,15.0,10.0,16.0,63.0,809.0,30.0,1391.0,2018
3,2018-M020,Brisbane,Hard,32,A,20180101,275,200282,NaN,WC,...,33.0,17.0,11.0,4.0,6.0,208.0,245.0,44.0,1055.0,2018
4,2018-M020,Brisbane,Hard,32,A,20180101,276,111581,NaN,Q,...,28.0,5.0,9.0,0.0,2.0,175.0,299.0,68.0,755.0,2018


In [5]:
final_df.describe()

,draw_size,tourney_date,match_num,winner_id,winner_ht,winner_age,loser_id,loser_ht,loser_age,best_of,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year
count,11823.000000,1.182300e+04,11823.000000,11823.000000,11739.000000,11822.000000,11823.000000,11593.000000,11822.000000,11823.000000,...,11389.000000,11389.000000,11390.000000,11389.000000,11389.000000,11785.000000,11785.000000,11685.000000,11685.000000,11823.000000
mean,60.271336,2.019878e+07,262.492261,120684.644845,187.539484,27.232093,120406.206969,186.652635,27.465970,3.396346,...,33.950654,14.813153,12.544864,4.828431,8.592063,71.396945,1888.729996,101.171673,1197.572871,2019.829654
std,42.579000,1.442071e+04,162.668020,29758.553485,7.091715,4.726342,29747.927903,6.892000,4.690963,0.797280,...,14.571416,6.946332,4.254158,3.294453,4.160238,115.403786,2125.416320,151.615237,1263.425190,1.439202
min,2.000000,2.018010e+07,1.000000,100644.000000,163.000000,14.900000,100644.000000,170.000000,14.500000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2018.000000
25%,32.000000,2.018123e+07,225.500000,105173.000000,183.000000,23.400000,105311.000000,183.000000,23.700000,3.000000,...,23.000000,10.000000,9.000000,2.000000,6.000000,19.000000,729.000000,35.000000,580.000000,2019.000000
50%,32.000000,2.020012e+07,277.000000,106045.000000,188.000000,26.900000,106043.000000,185.000000,27.400000,3.000000,...,32.000000,14.000000,12.000000,4.000000,8.000000,46.000000,1101.000000,66.000000,880.000000,2020.000000
75%,128.000000,2.021072e+07,290.000000,126138.500000,193.000000,30.900000,125802.000000,190.000000,30.800000,3.000000,...,42.000000,19.000000,15.000000,7.000000,11.000000,84.000000,2045.000000,107.000000,1341.000000,2021.000000
max,128.000000,2.022072e+07,1701.000000,211346.000000,211.000000,42.300000,211346.000000,211.000000,42.500000,5.000000,...,171.000000,50.000000,50.000000,27.000000,38.000000,1821.000000,12415.000000,2146.000000,12355.000000,2022.000000


In [6]:
# nr stands for not ranked
final_df['winner_seed'] = final_df['winner_seed'].replace(np.nan, '0')

In [7]:
final_df['loser_seed'] = final_df['loser_seed'].replace(np.nan, '0')

In [8]:
final_df['winner_seed'].value_counts()

0       6956
1.0      515
2.0      447
3.0      368
5.0      348
        ... 
24         3
33         3
16         3
33.0       2
LL         1
Name: winner_seed, Length: 70, dtype: int64

In [9]:
final_df['winner_seed'].replace('WC', '0', inplace = True)

In [10]:
final_df['loser_seed'].replace('WC', '0', inplace = True)

In [11]:
final_df[final_df['winner_seed'] == 'WC']

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year


In [12]:
final_df['winner_seed'].replace('LL', '0', inplace = True)

In [13]:
final_df['loser_seed'].replace('LL', '0', inplace = True)

In [14]:
final_df[final_df['winner_seed'] == 'LL']

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year


In [15]:
final_df['winner_seed'].replace('Q', '0', inplace = True)

In [16]:
final_df['loser_seed'].replace('Q', '0', inplace = True)

In [17]:
final_df[final_df['winner_seed'] == 'Q']

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year


In [18]:
len(final_df['winner_seed'])

11823

In [19]:
final_df['winner_seed'] = final_df['winner_seed'].astype(int)

In [20]:
final_df['loser_seed'] = final_df['loser_seed'].astype(int)

In [21]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11823 entries, 0 to 1925
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          11823 non-null  object 
 1   tourney_name        11823 non-null  object 
 2   surface             11823 non-null  object 
 3   draw_size           11823 non-null  int64  
 4   tourney_level       11823 non-null  object 
 5   tourney_date        11823 non-null  int64  
 6   match_num           11823 non-null  int64  
 7   winner_id           11823 non-null  int64  
 8   winner_seed         11823 non-null  int64  
 9   winner_entry        1811 non-null   object 
 10  winner_name         11823 non-null  object 
 11  winner_hand         11821 non-null  object 
 12  winner_ht           11739 non-null  float64
 13  winner_ioc          11823 non-null  object 
 14  winner_age          11822 non-null  float64
 15  loser_id            11823 non-null  int64  
 16  loser

In [22]:
# Next we are going to look at winner and loser entry
# I am assuming that nan players are just normal unseeded players and will also be called nr
final_df['winner_entry'].unique()

array([nan, 'WC', 'Q', 'LL', 'SE', 'PR', 'ALT', 'Alt'], dtype=object)

In [23]:
final_df['winner_entry'] = final_df['winner_entry'].replace(np.nan, '0')

In [24]:
final_df['winner_entry'].unique()

array(['0', 'WC', 'Q', 'LL', 'SE', 'PR', 'ALT', 'Alt'], dtype=object)

In [25]:
final_df['loser_entry'] = final_df['loser_entry'].replace(np.nan, '0')

In [26]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11823 entries, 0 to 1925
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          11823 non-null  object 
 1   tourney_name        11823 non-null  object 
 2   surface             11823 non-null  object 
 3   draw_size           11823 non-null  int64  
 4   tourney_level       11823 non-null  object 
 5   tourney_date        11823 non-null  int64  
 6   match_num           11823 non-null  int64  
 7   winner_id           11823 non-null  int64  
 8   winner_seed         11823 non-null  int64  
 9   winner_entry        11823 non-null  object 
 10  winner_name         11823 non-null  object 
 11  winner_hand         11821 non-null  object 
 12  winner_ht           11739 non-null  float64
 13  winner_ioc          11823 non-null  object 
 14  winner_age          11822 non-null  float64
 15  loser_id            11823 non-null  int64  
 16  loser

In [27]:
replace = final_df['winner_ht'].fillna(final_df['winner_ht'].mode()[0], inplace = True)

In [28]:
replace0 = final_df['loser_ht'].fillna(final_df['loser_ht'].mode()[0], inplace = True)

In [29]:
#Winner and loser age
#one null in each so just fill with mode
replace2 = final_df['winner_age'].fillna(final_df['winner_age'].mode()[0], inplace = True)

In [30]:
replace3 = final_df['loser_age'].fillna(final_df['loser_age'].mode()[0], inplace = True)

In [31]:
replace4 = final_df['winner_hand'].fillna(final_df['winner_hand'].mode()[0], inplace = True)

In [32]:
replace5 = final_df['loser_hand'].fillna(final_df['loser_hand'].mode()[0], inplace = True)

In [33]:
final_df = final_df.dropna()

In [34]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11266 entries, 0 to 1923
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          11266 non-null  object 
 1   tourney_name        11266 non-null  object 
 2   surface             11266 non-null  object 
 3   draw_size           11266 non-null  int64  
 4   tourney_level       11266 non-null  object 
 5   tourney_date        11266 non-null  int64  
 6   match_num           11266 non-null  int64  
 7   winner_id           11266 non-null  int64  
 8   winner_seed         11266 non-null  int64  
 9   winner_entry        11266 non-null  object 
 10  winner_name         11266 non-null  object 
 11  winner_hand         11266 non-null  object 
 12  winner_ht           11266 non-null  float64
 13  winner_ioc          11266 non-null  object 
 14  winner_age          11266 non-null  float64
 15  loser_id            11266 non-null  int64  
 16  loser

In [35]:
final_df['surface'].value_counts()

Hard     6448
Clay     3554
Grass    1264
Name: surface, dtype: int64

In [36]:
final_df["surface"] = final_df["surface"].astype('category')
final_df.dtypes

tourney_id              object
tourney_name            object
surface               category
draw_size                int64
tourney_level           object
tourney_date             int64
match_num                int64
winner_id                int64
winner_seed              int64
winner_entry            object
winner_name             object
winner_hand             object
winner_ht              float64
winner_ioc              object
winner_age             float64
loser_id                 int64
loser_seed               int64
loser_entry             object
loser_name              object
loser_hand              object
loser_ht               float64
loser_ioc               object
loser_age              float64
score                   object
best_of                  int64
round                   object
minutes                float64
w_ace                  float64
w_df                   float64
w_svpt                 float64
w_1stIn                float64
w_1stWon               float64
w_2ndWon

In [37]:
final_df["surface_cat"] = final_df["surface"].cat.codes
final_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat
0,2018-M020,Brisbane,Hard,32,A,20180101,271,105992,0,0,...,19.0,14.0,1.0,4.0,47.0,1010.0,52.0,909.0,2018,2
1,2018-M020,Brisbane,Hard,32,A,20180101,272,111577,0,0,...,7.0,9.0,7.0,11.0,54.0,890.0,94.0,593.0,2018,2
2,2018-M020,Brisbane,Hard,32,A,20180101,273,104797,0,0,...,29.0,15.0,10.0,16.0,63.0,809.0,30.0,1391.0,2018,2
3,2018-M020,Brisbane,Hard,32,A,20180101,275,200282,0,WC,...,17.0,11.0,4.0,6.0,208.0,245.0,44.0,1055.0,2018,2
4,2018-M020,Brisbane,Hard,32,A,20180101,276,111581,0,Q,...,5.0,9.0,0.0,2.0,175.0,299.0,68.0,755.0,2018,2


In [38]:
final_df['surface_cat'].value_counts()
#2 = Hard
#1 = Grass
#0 = Clay

2    6448
0    3554
1    1264
Name: surface_cat, dtype: int64

In [39]:
final_df['surface'].value_counts()


Hard     6448
Clay     3554
Grass    1264
Name: surface, dtype: int64

In [40]:
final_df = final_df.drop(columns = ['tourney_id', 'tourney_name', 'tourney_level', 'tourney_date', 'winner_ioc', 'loser_ioc',
                                   'match_num', 'winner_entry', 'loser_entry', 'surface'])
final_df.head()

,draw_size,winner_id,winner_seed,winner_name,winner_hand,winner_ht,winner_age,loser_id,loser_seed,loser_name,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat
0,32,105992,0,Ryan Harrison,R,183.0,25.6,104919,0,Leonardo Mayer,...,19.0,14.0,1.0,4.0,47.0,1010.0,52.0,909.0,2018,2
1,32,111577,0,Jared Donaldson,R,188.0,21.2,111442,0,Jordan Thompson,...,7.0,9.0,7.0,11.0,54.0,890.0,94.0,593.0,2018,2
2,32,104797,0,Denis Istomin,R,188.0,31.3,106000,7,Damir Dzumhur,...,29.0,15.0,10.0,16.0,63.0,809.0,30.0,1391.0,2018,2
3,32,200282,0,Alex De Minaur,R,183.0,18.8,105449,0,Steve Johnson,...,17.0,11.0,4.0,6.0,208.0,245.0,44.0,1055.0,2018,2
4,32,111581,0,Michael Mmoh,R,188.0,19.9,105643,0,Federico Delbonis,...,5.0,9.0,0.0,2.0,175.0,299.0,68.0,755.0,2018,2


In [41]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11266 entries, 0 to 1923
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   draw_size           11266 non-null  int64  
 1   winner_id           11266 non-null  int64  
 2   winner_seed         11266 non-null  int64  
 3   winner_name         11266 non-null  object 
 4   winner_hand         11266 non-null  object 
 5   winner_ht           11266 non-null  float64
 6   winner_age          11266 non-null  float64
 7   loser_id            11266 non-null  int64  
 8   loser_seed          11266 non-null  int64  
 9   loser_name          11266 non-null  object 
 10  loser_hand          11266 non-null  object 
 11  loser_ht            11266 non-null  float64
 12  loser_age           11266 non-null  float64
 13  score               11266 non-null  object 
 14  best_of             11266 non-null  int64  
 15  round               11266 non-null  object 
 16  minut

In [42]:
final_df.head()

,draw_size,winner_id,winner_seed,winner_name,winner_hand,winner_ht,winner_age,loser_id,loser_seed,loser_name,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat
0,32,105992,0,Ryan Harrison,R,183.0,25.6,104919,0,Leonardo Mayer,...,19.0,14.0,1.0,4.0,47.0,1010.0,52.0,909.0,2018,2
1,32,111577,0,Jared Donaldson,R,188.0,21.2,111442,0,Jordan Thompson,...,7.0,9.0,7.0,11.0,54.0,890.0,94.0,593.0,2018,2
2,32,104797,0,Denis Istomin,R,188.0,31.3,106000,7,Damir Dzumhur,...,29.0,15.0,10.0,16.0,63.0,809.0,30.0,1391.0,2018,2
3,32,200282,0,Alex De Minaur,R,183.0,18.8,105449,0,Steve Johnson,...,17.0,11.0,4.0,6.0,208.0,245.0,44.0,1055.0,2018,2
4,32,111581,0,Michael Mmoh,R,188.0,19.9,105643,0,Federico Delbonis,...,5.0,9.0,0.0,2.0,175.0,299.0,68.0,755.0,2018,2


In [43]:
final_df['winner_hand'].value_counts()

R    9696
L    1517
U      53
Name: winner_hand, dtype: int64

In [44]:
one_hot = pd.get_dummies(final_df, columns=["winner_hand"])

In [45]:
one_hot2 = pd.get_dummies(final_df, columns=["loser_hand"])

In [46]:
final_df.drop(columns= ['winner_hand'], axis=1)
final_df.merge(one_hot)

,draw_size,winner_id,winner_seed,winner_name,winner_hand,winner_ht,winner_age,loser_id,loser_seed,loser_name,...,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat,winner_hand_L,winner_hand_R,winner_hand_U
0,32,105992,0,Ryan Harrison,R,183.0,25.6,104919,0,Leonardo Mayer,...,4.0,47.0,1010.0,52.0,909.0,2018,2,0,1,0
1,32,111577,0,Jared Donaldson,R,188.0,21.2,111442,0,Jordan Thompson,...,11.0,54.0,890.0,94.0,593.0,2018,2,0,1,0
2,32,104797,0,Denis Istomin,R,188.0,31.3,106000,7,Damir Dzumhur,...,16.0,63.0,809.0,30.0,1391.0,2018,2,0,1,0
3,32,200282,0,Alex De Minaur,R,183.0,18.8,105449,0,Steve Johnson,...,6.0,208.0,245.0,44.0,1055.0,2018,2,0,1,0
4,32,111581,0,Michael Mmoh,R,188.0,19.9,105643,0,Federico Delbonis,...,2.0,175.0,299.0,68.0,755.0,2018,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11261,4,202120,0,Rowland Phillips,R,188.0,28.0,133018,0,Michail Pervolarakis,...,8.0,1390.0,4.0,402.0,98.0,2022,0,0,1,0
11262,4,209080,0,Blaise Bicknell,R,188.0,20.2,202065,0,Petros Tsitsipas,...,4.0,1130.0,8.0,808.0,23.0,2022,0,0,1,0
11263,4,209362,0,Aristotelis Thanos,R,188.0,20.8,209080,0,Blaise Bicknell,...,13.0,1103.0,9.0,1130.0,8.0,2022,0,0,1,0
11264,4,202065,0,Petros Tsitsipas,U,185.0,21.5,202120,0,Rowland Phillips,...,8.0,808.0,23.0,1390.0,4.0,2022,0,0,0,1


In [49]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11266 entries, 0 to 1923
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   draw_size           11266 non-null  int64  
 1   winner_id           11266 non-null  int64  
 2   winner_seed         11266 non-null  int64  
 3   winner_name         11266 non-null  object 
 4   winner_hand         11266 non-null  object 
 5   winner_ht           11266 non-null  float64
 6   winner_age          11266 non-null  float64
 7   loser_id            11266 non-null  int64  
 8   loser_seed          11266 non-null  int64  
 9   loser_name          11266 non-null  object 
 10  loser_hand          11266 non-null  object 
 11  loser_ht            11266 non-null  float64
 12  loser_age           11266 non-null  float64
 13  score               11266 non-null  object 
 14  best_of             11266 non-null  int64  
 15  round               11266 non-null  object 
 16  minut

In [51]:
len(final_df['winner_name'].unique())

413

In [79]:
final_df.drop(columns= ['winner_hand', 'loser_hand'])

,draw_size,winner_id,winner_seed,winner_name,winner_ht,winner_age,loser_id,loser_seed,loser_name,loser_ht,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat
0,32,105992,0,Ryan Harrison,183.0,25.6,104919,0,Leonardo Mayer,188.0,...,19.0,14.0,1.0,4.0,47.0,1010.0,52.0,909.0,2018,2
1,32,111577,0,Jared Donaldson,188.0,21.2,111442,0,Jordan Thompson,183.0,...,7.0,9.0,7.0,11.0,54.0,890.0,94.0,593.0,2018,2
2,32,104797,0,Denis Istomin,188.0,31.3,106000,7,Damir Dzumhur,172.0,...,29.0,15.0,10.0,16.0,63.0,809.0,30.0,1391.0,2018,2
3,32,200282,0,Alex De Minaur,183.0,18.8,105449,0,Steve Johnson,188.0,...,17.0,11.0,4.0,6.0,208.0,245.0,44.0,1055.0,2018,2
4,32,111581,0,Michael Mmoh,188.0,19.9,105643,0,Federico Delbonis,190.0,...,5.0,9.0,0.0,2.0,175.0,299.0,68.0,755.0,2018,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1919,4,202120,0,Rowland Phillips,188.0,28.0,133018,0,Michail Pervolarakis,193.0,...,20.0,13.0,4.0,8.0,1390.0,4.0,402.0,98.0,2022,0
1920,4,209080,0,Blaise Bicknell,188.0,20.2,202065,0,Petros Tsitsipas,185.0,...,14.0,10.0,0.0,4.0,1130.0,8.0,808.0,23.0,2022,0
1921,4,209362,0,Aristotelis Thanos,188.0,20.8,209080,0,Blaise Bicknell,188.0,...,12.0,10.0,11.0,13.0,1103.0,9.0,1130.0,8.0,2022,0
1922,4,202065,0,Petros Tsitsipas,185.0,21.5,202120,0,Rowland Phillips,188.0,...,20.0,15.0,4.0,8.0,808.0,23.0,1390.0,4.0,2022,0


## Average each column per player to find aggregate performance stats from 2016-current

In [83]:
11266* .05

563.3000000000001

In [82]:
#winners = list(final_df['winner_name'])

In [ ]:
#for name in winners:
    

In [ ]:
#categorical_features = ["winner_hand", "loser_hand"]
#categorical_transformer = OneHotEncoder(handle_unknown="ignore")

#preprocessor = ColumnTransformer(
 #   transformers= ["cat", categorical_transformer, categorical_features])

## Webscrape UTR

In [52]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [53]:
resp = requests.get('https://www.universaltennis.com/rankings')

In [58]:
resp.text

'<!DOCTYPE html><html><head><title data-react-helmet="true"></title><meta name="apple-itunes-app" content="app-id=1519232627"/><meta name="facebook-domain-verification" content="knhuo4oml6yt1hj91zq747g6afvu06"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="stylesheet" href="https://pro.fontawesome.com/releases/v5.7.2/css/all.css" integrity="sha384-6jHF7Z3XI3fF4XZixAuSu0gGKrXwoX/w3uFPxC56OtjChio7wtTGJWRW53Nhx6Ev" crossorigin="anonymous"/><link rel="stylesheet" type="text/css" href="/fonts/utr-fonts.css"/><link rel="stylesheet" type="text/css" href="/fonts/utr-fonts2.css"/><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/slick-carousel/1.6.0/slick.min.css" integrity="sha256-jySGIHdxeqZZvJ9SHgPNjbsBP8roij7/WjgkoGTJICk=" crossorigin="anonymous"/><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/slick-carousel/1.6.0/slick-theme.css" integrity="sha256-etrwgFLGpqD4oNAFW08ZH9Bzif5ByXK2lXNHKy7LQGo=" crossorigin="anonymous"/>

In [59]:
soup = BeautifulSoup(resp.text)

In [62]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title data-react-helmet="true">
  </title>
  <meta content="app-id=1519232627" name="apple-itunes-app"/>
  <meta content="knhuo4oml6yt1hj91zq747g6afvu06" name="facebook-domain-verification"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.7.2/css/all.css" integrity="sha384-6jHF7Z3XI3fF4XZixAuSu0gGKrXwoX/w3uFPxC56OtjChio7wtTGJWRW53Nhx6Ev" rel="stylesheet"/>
  <link href="/fonts/utr-fonts.css" rel="stylesheet" type="text/css"/>
  <link href="/fonts/utr-fonts2.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://cdnjs.cloudflare.com/ajax/libs/slick-carousel/1.6.0/slick.min.css" integrity="sha256-jySGIHdxeqZZvJ9SHgPNjbsBP8roij7/WjgkoGTJICk=" rel="stylesheet"/>
  <link crossorigin="anonymous" href="https://cdnjs.cloudflare.com/ajax/libs/slick-carousel/1.6.0/slick-theme.css" integrity="sha256-etrwgFLGpqD4oNAFW08ZH9Bzif5ByXK2

In [ ]:
#table = soup.find("table", attrs={"class":"wikitable"})

In [69]:
soup.find("div", class_='row style_list__5jpUl')

<div class="row style_list__5jpUl"></div>

In [72]:
soup.find_all('div', class_='RankingListV2_rankItem__2f-O1')

[]

In [73]:
pd.read_html('https://www.universaltennis.com/rankings')

ImportError: html5lib not found, please install it

In [ ]:
# Setting our X and y variables
X = final_df.drop(columns='winner_name')
y = final_df['winner_name']
# Preparing our hold out data for train test split
X_cols = X.columns
X, X_hold, y, y_hold = train_test_split(X, y, random_state=42, test_size=.05)
# Regular train test split
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=42)

In [ ]:
# Grouping categorical columns
cat_columns = [c for c in X_train.columns if X_train[c].dtype == 'O']
# Grouping numerical columns
num_columns = [c for c in X_train.columns if X_train[c].dtype in ['int64', 'float64']]
# Combining the numerical and categorical columns
used_cols = [*cat_columns, *num_columns]

In [ ]:
# Instantiating OneHotEncoder(ohe)
cat_transformer = OneHotEncoder(drop='first')
# Instantiating StandardScaler
num_transformer = StandardScaler()

In [ ]:
# Instantiating ColumnTransformer
preprocessor = ColumnTransformer([
    ('cats', cat_transformer, cat_columns),
    ('nums', num_transformer, num_columns)
])

In [ ]:
# Function to show train and test accuracy scores
def evaluate(model, Xtr, Xte, ytr, yte):
    train_preds = model.predict(Xtr)
    test_preds = model.predict(Xte)
    train_probas = model.predict_proba(Xtr)[:,1]
    test_probas {= model.predict_proba(Xte)[:,1]
    plot_confusion_matrix(model, Xte, yte)
    plot_roc_curve(model, Xte, yte)
    print(f"Train Accuracy: {accuracy_score(y_train, train_preds):.4f}")
    print("*" * 20)
    print(f"Test Accuracy: {accuracy_score(y_test, test_preds):.4f}")